In [2]:
#HolidayImageManipulator.ipynb
#Created by Noah Walsh, Ben Valois, Rick Djeuhon, Derek Windahl, Jake Hamilton
#Deep Learning Holiday Image Manipulation
#Based on Deep Dreaming in Keras from the keras team and alterations from Alan Jamieson
#Make sure to have holiday_weights downloaded or created by TrainHolidayV3.ipynb
#12/6/18

from __future__ import print_function

from keras.preprocessing.image import load_img, save_img, img_to_array
import numpy as np
import scipy

from keras.applications import inception_v3
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator

base_image_path = './Pittsburg_pic.jpg'
result_prefix = './thisresult1'

# These are the names of the layers
# for which we try to maximize activation,
# as well as their weight in the final loss
# we try to maximize.
# You can tweak these setting to obtain new visual effects.
settings = {
    'features': {
        'mixed2': 4, 
        'mixed3': 0,
        'mixed4': 4,
        'mixed5': 0,
    },
}
#Since InteceptionV3 for image manipulation has no fully connected layer, 
#pre-trained holiday weights need to be loaded in secondarily.
model2 = inception_v3.InceptionV3(weights=None,
                                 include_top=False)

#Load pre-trained weights for the convolutional layers.
model2.load_weights('holiday_weights.h5',by_name =True)
dream = model2.input
print('Model loaded.')

# Get the symbolic outputs of each "key" layer (we gave them unique names).
layer_dict = dict([(layer.name, layer) for layer in model2.layers])

# Define the loss.
loss = K.variable(0.)
for layer_name in settings['features']:
    # Add the L2 norm of the features of a layer to the loss.
    assert (layer_name in layer_dict.keys(),
            'Layer ' + layer_name + ' not found in model.')
    coeff = settings['features'][layer_name]
    x = layer_dict[layer_name].output
    # We avoid border artifacts by only involving non-border pixels in the loss.
    scaling = K.prod(K.cast(K.shape(x), 'float32'))
    if K.image_data_format() == 'channels_first':
        loss += coeff * K.sum(K.square(x[:, :, 2: -2, 2: -2])) / scaling
    else:
        loss += coeff * K.sum(K.square(x[:, 2: -2, 2: -2, :])) / scaling

# Compute the gradients of the dream wrt the loss.
grads = K.gradients(loss, dream)[0]
# Normalize gradients.
grads /= K.maximum(K.mean(K.abs(grads)), K.epsilon())

# Set up function to retrieve the value
# of the loss and gradients given an input image.
outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)

def preprocess_image(image_path):
    # Util function to open, resize and format pictures
    # into appropriate tensors.
    img = load_img(image_path)
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    return img


def deprocess_image(x):
    # Util function to convert a tensor into a valid image.
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((x.shape[1], x.shape[2], 3))
    x /= 2.
    x += 0.5
    x *= 255.
    x = np.clip(x, 0, 255).astype('uint8')
    return x

K.set_learning_phase(0)



def eval_loss_and_grads(x): #evaluate the loss and gradients
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values


def resize_img(img, size): #resize the image
    img = np.copy(img)
    if K.image_data_format() == 'channels_first':
        factors = (1, 1,
                   float(size[0]) / img.shape[2],
                   float(size[1]) / img.shape[3])
    else:
        factors = (1,
                   float(size[0]) / img.shape[1],
                   float(size[1]) / img.shape[2],
                   1)
    return scipy.ndimage.zoom(img, factors, order=1)


def gradient_ascent(x, iterations, step, max_loss=None): #Add gradients to the image to produce the manipulations.
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('..Loss value at', i, ':', loss_value)
        x += step * grad_values
    return x


"""Process:
- Load the original image.
- Define a number of processing scales (i.e. image shapes),
    from smallest to largest.
- Resize the original image to the smallest scale.
- For every scale, starting with the smallest (i.e. current one):
    - Run gradient ascent
    - Upscale image to the next scale
    - Reinject the detail that was lost at upscaling time
- Stop when we are back to the original size.
To obtain the detail lost during upscaling, we simply
take the original image, shrink it down, upscale it,
and compare the result to the (resized) original image.
"""


# Playing with these hyperparameters will also allow you to achieve new effects
step = 0.03  # Gradient ascent step size
num_octave = 4  # Number of scales at which to run gradient ascent
octave_scale = 1.6  # Size ratio between scales
iterations = 30  # Number of ascent steps per scale
max_loss = 40.

img = preprocess_image(base_image_path)
if K.image_data_format() == 'channels_first':
    original_shape = img.shape[2:]
else:
    original_shape = img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
    successive_shapes.append(shape)
successive_shapes = successive_shapes[::-1]
original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])

for shape in successive_shapes:
    print('Processing image shape', shape)
    img = resize_img(img, shape)
    img = gradient_ascent(img,
                          iterations=iterations,
                          step=step,
                          max_loss=max_loss)
    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
    same_size_original = resize_img(original_img, shape)
    lost_detail = same_size_original - upscaled_shrunk_original_img

    img += lost_detail
    print(img)
    shrunk_original_img = resize_img(original_img, shape)

print(result_prefix)
save_img(result_prefix + '.png', deprocess_image(np.copy(img)))
print(result_prefix,".png finised")

Model loaded.


<ipython-input-2-3cc1b7fb11e8>:49: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert (layer_name in layer_dict.keys(),


Processing image shape (131, 253)
..Loss value at 0 : 0.00024237491
..Loss value at 1 : 0.00033389305
..Loss value at 2 : 0.00046289965
..Loss value at 3 : 0.0006476549
..Loss value at 4 : 0.0009211738
..Loss value at 5 : 0.0013422704
..Loss value at 6 : 0.0019937556
..Loss value at 7 : 0.00292781
..Loss value at 8 : 0.0041645723
..Loss value at 9 : 0.0057314783
..Loss value at 10 : 0.0076404535
..Loss value at 11 : 0.0099412035
..Loss value at 12 : 0.012581385
..Loss value at 13 : 0.015761385
..Loss value at 14 : 0.019300155
..Loss value at 15 : 0.02332038
..Loss value at 16 : 0.027856378
..Loss value at 17 : 0.03294669
..Loss value at 18 : 0.038556326
..Loss value at 19 : 0.044778243
..Loss value at 20 : 0.051602826
..Loss value at 21 : 0.059007246
..Loss value at 22 : 0.06701027
..Loss value at 23 : 0.075643815
..Loss value at 24 : 0.08497107
..Loss value at 25 : 0.09485197
..Loss value at 26 : 0.105394304
..Loss value at 27 : 0.11651458
..Loss value at 28 : 0.12840275
..Loss value 

Processing image shape (539, 1038)
..Loss value at 0 : 0.10100711
..Loss value at 1 : 0.106164
..Loss value at 2 : 0.11143309
..Loss value at 3 : 0.11680238
..Loss value at 4 : 0.12231096
..Loss value at 5 : 0.12803714
..Loss value at 6 : 0.13389237
..Loss value at 7 : 0.13995233
..Loss value at 8 : 0.14623316
..Loss value at 9 : 0.1527301
..Loss value at 10 : 0.15947133
..Loss value at 11 : 0.16642407
..Loss value at 12 : 0.17363256
..Loss value at 13 : 0.18111181
..Loss value at 14 : 0.18890238
..Loss value at 15 : 0.19695258
..Loss value at 16 : 0.20523992
..Loss value at 17 : 0.21383908
..Loss value at 18 : 0.22279093
..Loss value at 19 : 0.23203579
..Loss value at 20 : 0.24159388
..Loss value at 21 : 0.25161237
..Loss value at 22 : 0.26194257
..Loss value at 23 : 0.27257577
..Loss value at 24 : 0.28363597
..Loss value at 25 : 0.29511926
..Loss value at 26 : 0.30704826
..Loss value at 27 : 0.31954604
..Loss value at 28 : 0.33249503
..Loss value at 29 : 0.34591842
[[[[ 0.17616843  0